# Interconnect

Horizontal beam terminated with an anchored pad. Routes electrical signals between MEMS components.

In [ ]:
import sys
sys.path.insert(0, "../../libraries")

from mcw_custom_optical_mems_pdk import PDK
PDK.activate()

from interconnect import make_interconnect

In [ ]:
ic = make_interconnect()
ic.plot()